In [29]:
import pandas as pd
import glob 

In [30]:
meteo_files = glob.glob("data/openmeteo_location*.{}".format("csv"))
meteo_joined = []

for file in meteo_files:
    df = pd.read_csv(file, parse_dates=["date"])
    df = df.set_index("date").resample('15min').ffill().reset_index()
    meteo_joined.append(df)

meteo_data = pd.concat(meteo_joined, ignore_index=True)

meteo_data["temperature_delta"] = meteo_data["temperature_2m"] - meteo_data["temperature_2m_previous_day1"]
meteo_data["direct_normal_irradiance_delta"] = meteo_data["direct_normal_irradiance_instant"] - meteo_data["direct_normal_irradiance_instant_previous_day1"]
meteo_data["shortwave_radiation_delta"] = meteo_data["shortwave_radiation"] - meteo_data["shortwave_radiation_previous_day1"]
meteo_data["wind_speed_80m_delta"] = meteo_data["wind_speed_80m"] - meteo_data["wind_speed_80m_previous_day1"]
meteo_data["wind_direction_80m_delta"] = meteo_data["wind_direction_80m"] - meteo_data["wind_direction_80m_previous_day1"]

meteo_data["date"] = meteo_data["date"].dt.tz_convert('EET')
meteo_data["date"] = meteo_data["date"].dt.tz_localize(None)
meteo_data = meteo_data.drop(columns=['Unnamed: 0'])


spot_balancing = pd.read_csv("data/spot_balancing_2025Q1.csv")
spot_balancing["date"] = pd.to_datetime(spot_balancing["date"], format="%Y-%m-%d %H:%M:%S")

spot_balancing = spot_balancing[["date", "LT_imb_MW", "LT_mfrr_SA_up_activ", 
                                 "LT_mfrr_SA_down_activ", "10YLT-1001A0008Q_DA_eurmwh"]]


merged_data = pd.merge(spot_balancing, meteo_data, on="date", how="inner")

merged_data.to_csv("merged_data.csv", index=False)

In [31]:
merged_data.columns


Index(['date', 'LT_imb_MW', 'LT_mfrr_SA_up_activ', 'LT_mfrr_SA_down_activ',
       '10YLT-1001A0008Q_DA_eurmwh', 'temperature_2m',
       'temperature_2m_previous_day1', 'direct_normal_irradiance_instant',
       'direct_normal_irradiance_instant_previous_day1', 'shortwave_radiation',
       'shortwave_radiation_previous_day1', 'wind_speed_80m',
       'wind_speed_80m_previous_day1', 'wind_direction_80m',
       'wind_direction_80m_previous_day1', 'latitude', 'longitude',
       'temperature_delta', 'direct_normal_irradiance_delta',
       'shortwave_radiation_delta', 'wind_speed_80m_delta',
       'wind_direction_80m_delta'],
      dtype='object')